In [66]:
# Cell 1: Setup and Imports
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from datetime import datetime

# Replace 'your_api_key' with your actual Token Terminal API key
API_KEY = '2acb79dd-f8ca-48b5-b10c-bbee2290ed94'
BASE_URL = 'https://api.tokenterminal.com/v2'

In [67]:
def fetch_projects():
    url = f'{BASE_URL}/projects'
    headers = {
        'Authorization': f'Bearer {API_KEY}'
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Raise an exception for HTTP errors
    return response.json()

def fetch_metrics():
    url = f'{BASE_URL}/metrics'
    headers = {
        'Authorization': f'Bearer {API_KEY}'
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Raise an exception for HTTP errors
    return response.json()

def fetch_time_series_data(metric_ids, project_ids, start_date, end_date):
    cache = {project_id: [] for project_id in project_ids}
    project_ids_str = ','.join(project_ids)
    
    for metric_id in metric_ids:
        url = f'{BASE_URL}/metrics/{metric_id}'
        headers = {
            'Authorization': f'Bearer {API_KEY}'
        }
        params = {
            'project_ids': project_ids_str,
            'start': start_date,
            'end': end_date
        }
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 404:
            print(f"Data for projects or metric ID '{metric_id}' not found.")
            continue
        response.raise_for_status()  # Raise an exception for other HTTP errors
        data = response.json()['data']
        
        for entry in data:
            cache[entry['project_id']].append(entry)
    
    for project_id in cache:
        if cache[project_id]:
            df = pd.DataFrame(cache[project_id])
            df['timestamp'] = pd.to_datetime(df['timestamp'])
            df = df.pivot(index='timestamp', columns='metric_id', values='value')
            df.reset_index(inplace=True)
            if 'market_cap_circulating' in df.columns and 'market_cap_fully_diluted' in df.columns:
                df['circulating_supply'] = df['market_cap_circulating'] / df['market_cap_fully_diluted']
            cache[project_id] = df
        else:
            cache[project_id] = pd.DataFrame()
    
    return cache


In [68]:
import plotly.graph_objs as go
import numpy as np
import pandas as pd

def plot_time_series(data_dict, project_id):
    fig = go.Figure()
    data = data_dict[project_id]
    if not data.empty:
        data.set_index('timestamp', inplace=True)
        for column in data.columns:
            fig.add_trace(go.Scatter(x=data.index, y=data[column], mode='lines+markers', name=column))
    
    fig.update_layout(
        title=f'Metrics Over Time for {project_id.capitalize()}',
        xaxis_title='Date',
        yaxis_title='Value',
        legend_title='Metric',
        template='plotly_white'
    )
    fig.show()

def plot_circulating_supply(data_dict):
    fig = go.Figure()

    for project_id, data in data_dict.items():
        if 'circulating_supply' in data.columns and 'market_cap_circulating' in data.columns and 'market_cap_fully_diluted' in data.columns:
            data.set_index('timestamp', inplace=True)
            
            # Add circulating supply data
            fig.add_trace(go.Scatter(
                x=data.index, y=data['circulating_supply'], mode='lines+markers', name=f'{project_id} Circulating Supply',
                yaxis='y1'
            ))
            
            # Add market cap circulating data
            fig.add_trace(go.Scatter(
                x=data.index, y=data['market_cap_circulating'], mode='lines+markers', name=f'{project_id} Market Cap Circulating',
                yaxis='y2'
            ))
            
            # Add market cap fully diluted data
            fig.add_trace(go.Scatter(
                x=data.index, y=data['market_cap_fully_diluted'], mode='lines+markers', name=f'{project_id} Market Cap Fully Diluted',
                yaxis='y2'
            ))

    fig.update_layout(
        title='Circulating Supply and Market Cap Over Time for Selected Projects',
        xaxis_title='Date',
        yaxis=dict(
            title='Circulating Supply',
            side='left'
        ),
        yaxis2=dict(
            title='Market Cap',
            side='right',
            overlaying='y'
        ),
        legend_title='Metric',
        template='plotly_white'
    )
    fig.show()

In [69]:
projects = fetch_projects()
metrics = fetch_metrics()

# Convert projects and metrics to DataFrame for easier manipulation
projects_df = pd.DataFrame(projects['data'])
metrics_df = pd.DataFrame(metrics['data'])

metric_ids = ['price','market_cap_circulating','market_cap_fully_diluted']

# Limit to a subset of project IDs
project_ids = ['ethereum','internetcomputer','bitcoin','ripple','solana','dogecoin','cardano','chainlink','near-protocol','thorchain','polygon','avalanche','binance-smart-chain','polkadot','litecoin','hedera']  # Example project IDs
start_date = '2022-01-01'
end_date = '2024-5-28'

# Fetch time series data for the given metrics and project IDs
time_series_data = fetch_time_series_data(metric_ids, project_ids, start_date, end_date)

In [70]:
plot_circulating_supply(time_series_data)